In [ ]:
# Extraer las intervenciones del texto de los PDFs y escribirlas en TXT
import os, re
from PyPDF2 import PdfReader

basedir = '/workspace/nemo/TFM/'

# Ubicación de los PDF con las transcripciones
pdf_folder = basedir + 'datasets/automated-medical-transcription/transcripts/source/'
pdf_files = []

# Lista de PDFs
for file_path in sorted(os.listdir(pdf_folder)):
    if os.path.isfile(os.path.join(pdf_folder, file_path)) and file_path.endswith('.pdf'):
        pdf_files.append(os.path.join(pdf_folder, file_path))
        
for pdf_file in pdf_files:
    orig_name = os.path.splitext(os.path.basename(pdf_file))[0]
    print("FILE:", orig_name)
    
    file_out = open(pdf_folder + orig_name + ".txt", "a")
    # Extraemos el texto
    pdf_text = ''
    reader = PdfReader(pdf_file)
    for page in reader.pages:
        pdf_text += page.extract_text()

     # Quitamos los newline dentro de la misma frase
    filetext = re.sub("\n(?!Clinician: |Patient: )", " ", pdf_text)
    #TODO Tener en cuenta otro tipos de newline
    matches = re.finditer("(Clinician: |Patient: )\([^\)]+\) (.*)", filetext)
    num_matches = sum(1 for m in matches)
    matches = re.finditer("(Clinician: |Patient: )\([^\)]+\) (.*)", filetext)
    if num_matches == 0:
        filetext = re.sub("\n(?!CLIENT: |THERAPIST: )", " ", pdf_text)
        #TODO Tener en cuenta otro tipos de newline
        matches = re.finditer("(CLIENT: |THERAPIST: )(.*)", filetext)
        num_matches = sum(1 for m in matches)
        matches = re.finditer("(CLIENT: |THERAPIST: )(.*)", filetext)

    print('PDF extract: OK.', num_matches, "text lines\n")
    for match in matches:
        file_out.write(match.group(2) + '\n')

    file_out.close()
    
print('FIN!')

In [ ]:
# Dividir los archivos de audio según los silencios
import os, re, subprocess, glob
from mutagen.wave import WAVE

def split_audio(audiopath, dest_folder):
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)
    process = subprocess.run(['sox', audiopath, dest_folder + outwav_basename + '.wav', 
                              'silence', '-l', '1', '0.0', '-60d', '1', '1.5', '-60d', ':', 
                              'newfile', ':', 'restart'])
    return process.returncode

basedir = '/workspace/nemo/TFM/'

# Ubicación de los WAV
audio_orig_folder = basedir + 'datasets/automated-medical-transcription/recordings/'
audio_folder = basedir + 'tmp/'
wav_files = []
outwav_basename = 'split'

# Lista de WAVs
for file_path in sorted(os.listdir(audio_orig_folder)):
    if os.path.isfile(os.path.join(audio_orig_folder, file_path)) and file_path.endswith('.wav'):
        wav_files.append(os.path.join(audio_orig_folder, file_path))

for wav_file in wav_files:
    orig_name = os.path.splitext(os.path.basename(wav_file))[0]
    wav_orig_filename = orig_name + '.wav'
    print("FILE:", wav_file)
    
    if split_audio(wav_file, audio_folder + orig_name + '/') > 0:
        print('Error splitting audio')
    else:
        print('Split audio: OK.', len(glob.glob(audio_folder + orig_name + '/*.wav')), 'chunks')
        
        

Antes de ejecutar la siguiente celda hay que revisar los audios creados y el texto extraído

In [1]:
# Creamos el archivo manifest haciendo corresponder secuencialmente 
# los audios creados con las frases extraídas de los PDF
import os
from mutagen.wave import WAVE
from tools import prepare_text

basedir = '/workspace/nemo/TFM/'

# Ubicación de los WAV y las transcripciones
audio_folder = basedir + 'datasets/automated-medical-transcription/TFM/'
transcript_folder = basedir + 'datasets/automated-medical-transcription/transcripts/source/'
outwav_basename = 'splito'

fout_train = open(basedir + "data_manifest/amt_train_metafile.json", "a")
fout_val = open(basedir + "data_manifest/amt_val_metafile.json", "a")
fout_test = open(basedir + "data_manifest/amt_test_metafile.json", "a")

global_count = 0
split_val = 850
split_test = 1150
fout = fout_train
# Lista de carpetas con audios
for foldername in sorted(os.listdir(audio_folder)):
    current_audio_folder = os.path.join(audio_folder, foldername)
    if os.path.isdir(current_audio_folder) and not foldername.startswith("."):
        transcript_file = transcript_folder + foldername + '.txt'
        with open(transcript_file) as tfile:
            count = 1
            for line in tfile:
                if global_count == split_val: fout = fout_val
                if global_count == split_test: fout = fout_test
                current_wav = outwav_basename + str(count).zfill(3) + '.wav'
                audiopath = os.path.join(current_audio_folder, current_wav)
                
                if os.path.exists(audiopath):
                    audio = WAVE(audiopath)
                    audio_length = str(audio.info.length)
                else:
                    audio_length = '0'
                
                fout.write('{"audio_filepath": "' + audiopath + '", "duration": ' + audio_length + 
                           ', "text": "' + prepare_text(line) + '"}\n')
                count += 1
                global_count += 1

fout_train.close()
fout_val.close()
fout_test.close()

